In [ ]:
%matplotlib widget

In [ ]:
import h5py
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from eidein import Eidein

In [ ]:
with h5py.File("data/human_labeller.hdf5", "r") as datafile:
    X = datafile["X"][:]
    y = datafile["y"][:]
X.shape, y.shape

In [ ]:
def plot_mnist(ax, identifier, x, y, label):
    ax.imshow(x.reshape(32, 32), cmap='gray')
    ax.set_title("{}".format(y, label))

In [ ]:
i = 1

In [ ]:
with h5py.File("data/human_labeller.hdf5", "r") as datafile:
    index_query = datafile["index_query_{}".format(i)][:]
    entr_query = datafile["entr_query_{}".format(i)][:]
    logit_query = datafile["logit_query_{}".format(i)][:]
    # Python has different indexing then Julia.
    index_query -= 1

X_query = X[index_query]
y_query = y[index_query]

X_query.shape, y_query.shape, index_query.shape, entr_query.shape

In [ ]:
label_widget = ipywidgets.RadioButtons(options=np.arange(1, 11))

eidein = Eidein(
    index_query, logit_query, y_query, entr_query, X_query,
    plot_mnist, label_widget)
display(eidein)

In [ ]:
len(eidein.labelled), eidein.labelled

In [ ]:
with h5py.File("data/human_labeller.hdf5", "r+") as datafile:
    index_label = np.array(list(eidein.labelled.keys()))
    index_label += 1
    datafile.create_dataset("index_label_{}".format(i), data=index_label)